In [1]:
import os
import pandas as pd
import soundfile as sf
import numpy as np
import librosa
from scipy.stats import gmean
from tqdm.notebook import tqdm
import warnings
import gc

warnings.filterwarnings('ignore')

# ==================== ПАРАМЕТРЫ ====================
WIN_LEN = 20
HOP_LEN = 12
SAMPLE_RATE = 16000
HOP_LENGTH = 160
N_FFT = 512

BIN_MIN = int(2000 / (SAMPLE_RATE / N_FFT))      # ~64
BIN_MAX = int(3000 / (SAMPLE_RATE / N_FFT))      # ~96

# Валидные типы неисправностей (как в мел-датасете)
VALID_FAULTS = {'MF1', 'MF2', 'MF3', 'MF4', 'PC1', 'PC2', 'PC3', 'PC4', 'N'}

def parse_filename(filename):
    """
    Парсит имя файла и возвращает model_type, maneuvering_direction, fault.
    Формат имени: MODEL_MANEUVER_FAULT_... .wav
    """
    basename = filename.replace('.wav', '')
    parts = basename.split('_')
    
    model_type = 'unknown'
    maneuvering_direction = 'unknown'
    fault = 'unknown'
    
    if len(parts) >= 3:
        model_type = parts[0]
        maneuvering_direction = parts[1]
        fault_candidate = parts[2]
        
        if fault_candidate in VALID_FAULTS:
            fault = fault_candidate
        else:
            # Если не на третьем месте, ищем глубже
            for i, part in enumerate(parts[2:], start=2):
                if part in VALID_FAULTS:
                    fault = part
                    break
    return model_type, maneuvering_direction, fault

def extract_frame_features(y, sr):
    """Извлекает покадровые признаки (8 штук) из аудиосигнала"""
    stft_matrix = librosa.stft(y, n_fft=N_FFT, hop_length=HOP_LENGTH)
    power_spec = np.abs(stft_matrix)**2
    
    # Узкополосные признаки (2-3 кГц)
    band_spec = power_spec[BIN_MIN:BIN_MAX, :]
    band_energy = np.sum(band_spec, axis=0)
    
    amean = np.mean(band_spec + 1e-10, axis=0)
    g_mean = gmean(band_spec + 1e-10, axis=0)
    band_flatness = g_mean / amean
    
    peak_bins = np.argmax(band_spec, axis=0) + BIN_MIN
    band_peak_freq = peak_bins * (sr / N_FFT)
    
    # Широкополосные признаки
    zcr = librosa.feature.zero_crossing_rate(y, frame_length=N_FFT, hop_length=HOP_LENGTH)[0]
    centroid = librosa.feature.spectral_centroid(S=np.abs(stft_matrix), sr=sr)[0]
    bandwidth = librosa.feature.spectral_bandwidth(S=np.abs(stft_matrix), sr=sr)[0]
    rolloff = librosa.feature.spectral_rolloff(S=np.abs(stft_matrix), sr=sr, roll_percent=0.85)[0]
    rms = librosa.feature.rms(y=y, frame_length=N_FFT, hop_length=HOP_LENGTH)[0]
    
    min_len = min(len(band_energy), len(zcr), len(centroid), len(bandwidth), len(rolloff), len(rms))
    
    frame_features = np.vstack([
        band_energy[:min_len], band_flatness[:min_len], band_peak_freq[:min_len],
        zcr[:min_len], centroid[:min_len], bandwidth[:min_len], 
        rolloff[:min_len], rms[:min_len]
    ]).T  
    
    return frame_features

def split_and_aggregate(frame_features, win_len=WIN_LEN, hop_len=HOP_LEN):
    """Разбивает покадровые признаки на окна и вычисляет статистики"""
    if frame_features.shape[0] < win_len:
        padding = win_len - frame_features.shape[0]
        frame_features = np.pad(frame_features, ((0, padding), (0, 0)), mode='constant')
        
    N = frame_features.shape[0]
    windows_stats = []
    start = 0
    base_names = ['band_energy', 'band_flatness', 'band_freq', 'zcr', 'centroid', 'bandwidth', 'rolloff', 'rms']
    
    while start + win_len <= N:
        window = frame_features[start:start + win_len]
        
        w_mean = np.mean(window, axis=0)
        w_std = np.std(window, axis=0)
        w_max = np.max(window, axis=0)
        w_min = np.min(window, axis=0)
        
        window_combined = np.concatenate([w_mean, w_std, w_max, w_min])
        windows_stats.append(window_combined)
        start += hop_len
        
    col_names = []
    for stat in ['mean', 'std', 'max', 'min']:
        for name in base_names:
            col_names.append(f"custom_{name}_{stat}")
            
    return windows_stats, col_names

def process_file_custom(audio_path):
    """Обрабатывает один аудиофайл: извлекает признаки, добавляет метаданные"""
    try:
        data, sr = sf.read(audio_path)
        
        if len(data.shape) > 1:
            data = np.mean(data, axis=1)
            
        if sr != SAMPLE_RATE:
            data = librosa.resample(data, orig_sr=sr, target_sr=SAMPLE_RATE)
            
        frame_feats = extract_frame_features(data, SAMPLE_RATE)
        windows, col_names = split_and_aggregate(frame_feats)
        
        filename = os.path.basename(audio_path)
        
        # Парсим метки из имени файла
        model_type, maneuvering_direction, fault = parse_filename(filename)
        
        if windows:
            df = pd.DataFrame(windows, columns=col_names)
            df['filename'] = filename
            df['window_id'] = range(len(df))          # порядковый номер окна в файле
            df['model_type'] = model_type
            df['maneuvering_direction'] = maneuvering_direction
            df['fault'] = fault
            return df, None 
        return None, "Не удалось создать окна"
        
    except Exception as e:
        return None, str(e)

def process_single_folder(folder_path, folder_id, output_dir):
    """Обрабатывает одну папку с учётом кэширования"""
    cache_file = os.path.join(output_dir, "by_folder", f"{folder_id}.parquet")
    
    # Если уже обработано – загружаем из кэша
    if os.path.exists(cache_file):
        print(f"⏭️ Папка {folder_id} уже обработана. Загружаем из кэша...")
        return pd.read_parquet(cache_file), []
    
    print(f"\n📁 Обработка папки: {folder_id}")
    
    wav_files = [f for f in os.listdir(folder_path) if f.lower().endswith('.wav')]
    if not wav_files:
        print("   ⚠️ Нет WAV файлов.")
        return None, []
        
    folder_dfs = []
    folder_bad_files = []
    
    for wav_file in tqdm(wav_files, desc=f"Файлы в {folder_id}", leave=False):
        full_path = os.path.join(folder_path, wav_file)
        df, error_msg = process_file_custom(full_path)
        
        if df is not None:
            folder_dfs.append(df)
        else:
            folder_bad_files.append({"filename": wav_file, "folder": folder_id, "error": error_msg})
            
    if folder_dfs:
        folder_combined_df = pd.concat(folder_dfs, ignore_index=True)
        folder_combined_df.to_parquet(cache_file, engine='pyarrow', compression='snappy', index=False)
        print(f"   ✅ Сохранено в кэш: {len(folder_combined_df)} окон.")
        return folder_combined_df, folder_bad_files
    else:
        print("   ❌ Не удалось извлечь признаки из файлов папки.")
        return None, folder_bad_files

# ==================== ОСНОВНОЙ БЛОК ====================
audio_folder = r"C:\Users\Е5\Documents\olesya\vkr\drone"
output_dir = os.path.join(audio_folder, "features_win20_hop12_stft_feat32")

# Создаём структуру папок
os.makedirs(output_dir, exist_ok=True)
os.makedirs(os.path.join(output_dir, "by_folder"), exist_ok=True)
os.makedirs(os.path.join(output_dir, "combined"), exist_ok=True)

all_dfs = []
all_bad_files = [] 

# Собираем все папки для обработки (как в мел-скрипте)
folders_to_process = []
for main_folder in ['A', 'B', 'C']:
    main_path = os.path.join(audio_folder, main_folder)
    if not os.path.exists(main_path): continue
        
    for data_split in ['train', 'valid', 'test']:
        split_path = os.path.join(main_path, data_split)
        if not os.path.exists(split_path): continue
            
        for mic_folder in ['mic1', 'mic2']:
            mic_path = os.path.join(split_path, mic_folder)
            if not os.path.exists(mic_path): continue
                
            folder_id = f"{main_folder}_{data_split}_{mic_folder}"
            folders_to_process.append({
                'path': mic_path,
                'id': folder_id
            })

print(f"🚀 Найдено папок для обработки: {len(folders_to_process)}")

# Обрабатываем каждую папку
for folder_info in folders_to_process:
    df, bad_files = process_single_folder(folder_info['path'], folder_info['id'], output_dir)
    
    if df is not None:
        all_dfs.append(df)
    if bad_files:
        all_bad_files.extend(bad_files)
        
    gc.collect()

# Финальное объединение
if all_dfs:
    print("\n🔄 Создание финального объединенного файла...")
    final_combined_df = pd.concat(all_dfs, ignore_index=True)
    
    out_file = os.path.join(output_dir, "combined", "custom_data_v1.parquet")
    final_combined_df.to_parquet(out_file, engine='pyarrow', compression='snappy', index=False)
    
    print(f"✅ УСПЕШНО ЗАВЕРШЕНО!")
    print(f"💾 Итоговый размер: {len(final_combined_df)} окон.")
    print(f"📁 Путь: {out_file}")
    
    if all_bad_files:
        print(f"\n⚠️ Внимание! Найдено битых файлов: {len(all_bad_files)}")
        for bf in all_bad_files[:5]:
            print(f" - [{bf['folder']}] {bf['filename']}: {bf['error'][:50]}...")
else:
    print("❌ Нет данных для объединения.")

🚀 Найдено папок для обработки: 18

📁 Обработка папки: A_train_mic1


Файлы в A_train_mic1:   0%|          | 0/32400 [00:00<?, ?it/s]

   ✅ Сохранено в кэш: 97200 окон.

📁 Обработка папки: A_train_mic2


Файлы в A_train_mic2:   0%|          | 0/32400 [00:00<?, ?it/s]

   ✅ Сохранено в кэш: 97200 окон.

📁 Обработка папки: A_valid_mic1


...

   ✅ Сохранено в кэш: 32400 окон.

📁 Обработка папки: C_test_mic1


Файлы в C_test_mic1:   0%|          | 0/10800 [00:00<?, ?it/s]

   ✅ Сохранено в кэш: 32400 окон.

📁 Обработка папки: C_test_mic2


Файлы в C_test_mic2:   0%|          | 0/10800 [00:00<?, ?it/s]

   ✅ Сохранено в кэш: 32400 окон.

🔄 Создание финального объединенного файла...
✅ УСПЕШНО ЗАВЕРШЕНО!
💾 Итоговый размер: 971721 окон.
📁 Путь: C:\Users\Е5\Documents\olesya\vkr\drone\features_win20_hop12_stft_feat32\combined\custom_data_v1.parquet

⚠️ Внимание! Найдено битых файлов: 93
 - [A_valid_mic1] A_B_MF2_788_WestDoor_728_snr=10.737970939546484.wav: Internal psf_fseek() failed....
 - [A_valid_mic1] A_B_MF3_719_SportsComplex_361_snr=12.020708415650434.wav: Error : unknown error in flac decoder....
 - [A_valid_mic1] A_B_MF3_768_ConstructionSite_286_snr=10.842793445321204.wav: Error : unknown error in flac decoder....
 - [A_valid_mic1] A_B_PC2_731_ConstructionSite_253_snr=11.166238417763065.wav: Error : unknown error in flac decoder....
 - [A_valid_mic1] A_B_PC4_651_WestDoor_535_snr=11.978523582907995.wav: Internal psf_fseek() failed....
